In [ ]:
guardian_key='your_key'
import json

In [ ]:
import requests
url='https://content.guardianapis.com/search?page=1&page-size=200&api-key=your_key'

In [ ]:
#testing response 
response=requests.get(url)

In [ ]:
#loading into json
guardian_response=response.json()

In [ ]:
result_list=guardian_response['response']['results']

In [ ]:
#accessing headline
result_list[0]['webTitle']

'Burnley v Watford: Premier League – live!'

In [ ]:
#accessing date on test
result_list[0]['webPublicationDate']

'2020-06-25T18:41:01Z'

In [ ]:
#test to pull multiple pages
# import requests

# url='https://content.guardianapis.com/search?'
# params = {
#     "api-key": "",
#     "page-size": "200" }

# #cur=0
# results=[]
# for pi in range(1,16):
#     params["page"] = pi
#     results.append(requests.get(url, params))
 

In [ ]:
# results_dicts = [] 

# for ele in range(len(results)):
#     results_dicts.append(results[ele].json())

In [ ]:
#results_dicts

In [ ]:
#setting up function for guardian api call
def guardian_call(params):
    response = requests.get(url, params)
    return json.loads(response.text)

articles = guardian_call(params)

In [ ]:
#test
articles['response']['results'][0]

{'id': 'science/2020/jun/12/aubrey-burl-obituary',
 'type': 'article',
 'sectionId': 'books',
 'sectionName': 'Books',
 'webPublicationDate': '2020-06-12T16:56:58Z',
 'webTitle': 'Aubrey Burl obituary',
 'webUrl': 'https://www.theguardian.com/science/2020/jun/12/aubrey-burl-obituary',
 'apiUrl': 'https://content.guardianapis.com/science/2020/jun/12/aubrey-burl-obituary',
 'isHosted': False,
 'pillarId': 'pillar/arts',
 'pillarName': 'Arts'}

In [ ]:
#setting up function to parse results
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    parsed_result=[]
    for article in results:
        article_list = [article['webTitle'],article['webPublicationDate']]
        parsed_result.append(article_list)
        
            
    return parsed_result

In [ ]:
#test
x=parse_results(articles['response']['results'])

In [ ]:
#test, visualize api results in DF
import pandas as pd
pd.DataFrame(x).head()

,0,1
0,Aubrey Burl obituary,2020-06-12T16:56:58Z
1,Belgian transport minister in row over 34-mile...,2020-06-12T16:48:58Z
2,Amusement arcades furious after No 10 blocks r...,2020-06-12T16:44:33Z
3,JK Rowling: UK domestic abuse adviser writes t...,2020-06-12T16:43:50Z
4,London fashion week drops elitist traditions a...,2020-06-12T16:42:57Z


In [ ]:
#creating a blank DF
# Write a function to take your parsed data and insert it into CSV
columns = ['text','date']
df = pd.DataFrame(columns=columns) #blank df
df.to_csv('guardian_headlines.csv')

In [ ]:
#creating a save function
def data_save(parsed_results, csv_filename):
    # your code to save the current results with all of the other results. 
    # I would save the data every time you pull 50 results
    # in case something breaks in the process.
    #reads in blank csv
    existing=pd.read_csv(csv_filename,index_col=0)
    #50 at a time DF
    new = pd.DataFrame(parsed_results,columns=columns)
    df = pd.concat([existing,new])
    df.to_csv(csv_filename)

In [ ]:
#creating a script for calling from API, parsing results, saving results for each page
# Write a script that combines the three functions above into a single process.

# create a variable  to keep track of which result you are in. 
cur = 16

#set up a while loop to go through and grab the result 
while cur < 19:
    #set the offset parameter to be where you currently are in the results 
    params['page'] = cur
    #make your API call with the new offset number
    results = guardian_call(params)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results['response']['results'])
    
    
    # use your function to insert your parsed results into the db
    data_save(parsed_results, 'guardian_headlines.csv')
    #increment the counter by 50 to move on to the next results
    cur += 1

In [ ]:
guardian_df = pd.read_csv('guardian_headlines.csv',index_col=0)

In [ ]:
guardian_df

,text,date
0,Family court judges given power to intervene i...,2020-06-24T23:01:29Z
1,Children at RNIB schools and homes put at risk...,2020-06-24T23:01:29Z
2,Child abuse victims to sue Celtic FC in landma...,2020-06-24T23:01:28Z
3,"Cryptic crossword No 28,169",2020-06-24T23:00:28Z
4,Trent Alexander-Arnold the spark for Liverpool...,2020-06-24T22:55:55Z
...,...,...
195,Jigsy review – Les Dennis sheds tears of a clo...,2020-06-09T13:01:21Z
196,What's the secret to great jollof rice? | Kitc...,2020-06-09T13:00:49Z
197,UK students: what impact is coronavirus having...,2020-06-09T12:39:15Z
198,IBM quits facial-recognition market over polic...,2020-06-09T12:37:27Z
